In [1]:
import pandas as pd
from sqlalchemy import create_engine
from config import user
from config import password
import numpy as np

In [2]:
tomato = "Data/rotten_tomatoes.csv"
imdb = "Data/IMDB_movies.csv"

In [3]:
tomato_df = pd.read_csv(tomato)
imdb_df = pd.read_csv(imdb)

In [4]:
tomato_df.head()

,Rank,Title,year,RatingTomatometer,No. of Reviews,Genres
0,1,Black Panther,2018),97,444,action|adventure
1,2,Mad Max: Fury Road,2015),97,394,action|adventure
2,3,Wonder Woman,2017),93,410,action|adventure
3,4,Metropolis,1927),99,118,action|adventure
4,5,Coco,2017),97,308,action|adventure


In [5]:
imdb_df.head(2)

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0429277,Zyzzyx Rd,Zyzzyx Rd,2006,2/24/06,"Crime, Drama, Thriller",90,USA,English,John Penney,...,"Leo Grillo, Katherine Heigl, Tom Sizemore, Ric...",The family man accountant Grant travels to Las...,4.1,983,"$2,000,000",30,30,NaN,11.0,2.0
1,tt4195920,Chicas paranoicas,Chicas paranoicas,2015,9/16/16,Comedy,100,Spain,Spanish,Pedro del Santo,...,"Patricia Valley, Mairen Muñoz, Marta Mir Martí...",'Chicas Paranoicas' is the first Spanish comed...,7.2,146,NaN,78,78,NaN,NaN,2.0


In [6]:
imdb_clean_df = imdb_df[['original_title', 'avg_vote','year','duration','country','usa_gross_income','worlwide_gross_income']]

In [7]:
tomato_clean_df = tomato_df.drop(columns='No. of Reviews')

In [8]:
tomato_clean_df['year'] = tomato_clean_df['year'].str.replace(')','')
tomato_clean_df['year'] = tomato_clean_df['year'].astype(int)

In [9]:
tomato_clean_df['Title'] = tomato_clean_df['Title'].str.strip().str.lower()

In [10]:
tomato_clean_df.head()

,Rank,Title,year,RatingTomatometer,Genres
0,1,black panther,2018,97,action|adventure
1,2,mad max: fury road,2015,97,action|adventure
2,3,wonder woman,2017,93,action|adventure
3,4,metropolis,1927,99,action|adventure
4,5,coco,2017,97,action|adventure


In [11]:
tomato_clean_df.dtypes

Rank                  int64
Title                object
year                  int64
RatingTomatometer     int64
Genres               object
dtype: object

In [12]:
imdb_clean_df = imdb_clean_df.rename(columns={"original_title": "title", "avg_vote": "user_score", "usa_gross_income": "usa_gross",
                                             "worlwide_gross_income":"world_gross"})

In [13]:
tomato_clean_df = tomato_clean_df.rename(columns={"Rank": "rank", "Title": "title", "RatingTomatometer": "rating", "Genres": "genres"})

In [14]:
clean_score = imdb_clean_df["user_score"]*10
imdb_clean_df['user_score'] = clean_score
imdb_clean_df.head()

,title,user_score,year,duration,country,usa_gross,world_gross
0,Zyzzyx Rd,41.0,2006,90,USA,30,30
1,Chicas paranoicas,72.0,2015,100,Spain,78,78
2,Perro come perro,67.0,2008,106,Colombia,80,80
3,The Objective,55.0,2008,90,"USA, Morocco",95,95
4,Dixie y la rebelión zombi,46.0,2014,82,Spain,120,120


In [15]:
imdb_clean_df['title'] = imdb_clean_df['title'].str.strip().str.lower()
imdb_clean_df.head()

,title,user_score,year,duration,country,usa_gross,world_gross
0,zyzzyx rd,41.0,2006,90,USA,30,30
1,chicas paranoicas,72.0,2015,100,Spain,78,78
2,perro come perro,67.0,2008,106,Colombia,80,80
3,the objective,55.0,2008,90,"USA, Morocco",95,95
4,dixie y la rebelión zombi,46.0,2014,82,Spain,120,120


In [16]:
imdb_clean_df.dtypes

title           object
user_score     float64
year             int64
duration         int64
country         object
usa_gross        int64
world_gross      int64
dtype: object

In [ ]:
imdb_clean_df.to_csv('imdb_clean.csv', index=False)

In [ ]:
tomato_clean_df.to_csv('tomato_clean.csv', index=False)

In [17]:
engine = create_engine('sqlite:///project2.db')
conn = engine.connect()

In [18]:
imdb_clean_df.to_sql('imdb', con=engine, index=False, if_exists='replace')

In [19]:
tomato_clean_df.to_sql('tomato', con=engine, index=False, if_exists='replace')

In [22]:
combined = pd.read_sql("""SELECT *
    FROM tomato AS t 
    JOIN imdb AS i ON 
    i.title = t.title""", conn)
combined

,rank,title,year,rating,genres,title,user_score,year,duration,country,usa_gross,world_gross
0,1,black panther,2018,97,action|adventure,black panther,73.0,2018,134,USA,700426566,1347280838
1,2,mad max: fury road,2015,97,action|adventure,mad max: fury road,81.0,2015,120,"Australia, South Africa",154058340,375225175
2,3,wonder woman,2017,93,action|adventure,wonder woman,74.0,2017,141,"USA, China, Hong Kong",412563408,821847012
3,4,metropolis,1927,99,action|adventure,metropolis,83.0,1927,153,Germany,1236166,1349711
4,5,coco,2017,97,action|adventure,coco,84.0,2017,105,USA,209726015,807083670
...,...,...,...,...,...,...,...,...,...,...,...,...
980,70,priest,2011,15,western,priest,70.0,1994,105,UK,4165845,4165845
981,71,american outlaws,2001,14,western,american outlaws,60.0,2001,94,USA,13342790,13678913
982,72,september dawn,2007,15,western,september dawn,58.0,2007,111,"USA, Canada",1066555,1066555
983,73,jonah hex,2010,12,western,jonah hex,47.0,2010,81,USA,10547117,10903312


In [ ]:
combined_cleaned = combined.drop_duplicates(subset=['title']) 
combined_cleaned

In [ ]:
combined_sorted = combined_cleaned.sort_values(['rating', 'user_score'], ascending=False)
combined_sorted.head(10)

In [ ]:
combined_sorted_imdb = combined_cleaned.sort_values(['user_score', 'rating'], ascending=False)
combined_sorted_imdb.head(10)

In [ ]:
combined_sorted_imdb = combined_cleaned.sort_values(['user_score', 'rating', 'usa_gross'], ascending=False)
combined_sorted_imdb.head(10)
